# Analysis - EV Market Trend Data

##### Dyab Asdi, Evan Xia, Raymond Sun

In [2]:
import pandas as pd
import numpy as np

### Pre-Process Data

In [7]:
# read data
df1 = pd.read_csv("Electric_Vehicle_Population_Data1.csv")
df2 = pd.read_csv("Electric_Vehicle_Population_Data2.csv")
df3 = pd.read_csv("Electric_Vehicle_Population_Data3.csv")

df = pd.concat([df1, df2, df3], ignore_index=True)

numerical_columns = ["Model Year", "Electric Range", "Base MSRP"]

print(df[numerical_columns].median())
df[numerical_columns].describe()

Model Year        2021.0
Electric Range       6.0
Base MSRP            0.0
dtype: float64


,Model Year,Electric Range,Base MSRP
count,495346.000000,495319.000000,495319.000000
mean,2020.409691,62.509219,1183.436422
std,3.128313,93.822478,8781.132275
min,1997.000000,0.000000,0.000000
25%,2018.000000,0.000000,0.000000
50%,2021.000000,6.000000,0.000000
75%,2023.000000,84.000000,0.000000
max,2025.000000,337.000000,845000.000000


In [10]:
numerical_columns.append("VIN (1-10)")
remaining_cols = list(set(df.columns) - set(numerical_columns))
df[remaining_cols].mode().iloc[0]
# df[remaining_cols].nunique()

State                                                                                               WA
2020 Census Tract                                                                        53033028200.0
Legislative District                                                                              41.0
Vehicle Location                                                           POINT (-122.13158 47.67858)
Clean Alternative Fuel Vehicle (CAFV) Eligibility    Eligibility unknown as battery range has not b...
DOL Vehicle ID                                                                                    4777
City                                                                                           Seattle
Electric Utility                                         PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
Postal Code                                                                                    98052.0
County                                                                   